In [1]:
import os
import re

import pandas as pd

from pymongo import MongoClient

In [2]:
'''
Configura a conexão com a base de dados mongoDB
'''
client = MongoClient()

client = MongoClient('localhost', 27800)

db = client.odp_logs

db_log_lines = db.log_lines

In [3]:
'''
Le as palavras a serem excluídas dos registros de log da rede
'''
web_words_content = []
with open('./res/web_words.txt') as wwf:
    for ww in wwf:
        web_words_content.insert(len(web_words_content), ww.strip())

In [4]:
'''
Le as palavras a serem excluídas como extensões das urls
'''
web_words_extensios = []
with open('./res/extensions.txt') as wwf:
    for ww in wwf:
        web_words_extensios.insert(len(web_words_extensios), ww.strip())

In [5]:
'''
Função criada para remover todos as caracteres que não sejam as letras do alfabeto
'''
def remove_special_characters(pstr):
    return re.sub('[^A-Za-z]+', '', pstr)

In [6]:
def remove_extensions(url):
    for ext in web_words_extensios:
        url = url.replace(ext, '')
    return url

In [7]:
'''
Normaliza a parte do hostname, efetuando a split por ponto, removendo as palavras comuns (www, com, br, net, etc) 
e depois removendo números e caracteres especiais
'''
def normalize_hostname(url):
    final_result = ''
    url = url.replace('http://', '').replace('https://', '')
    url_split = url.split('/')
    
    hostname_split = url_split[0].split('.')
    
    path_split = url_split[1:]

    for hs in hostname_split:
        normal_hs = remove_special_characters(hs)
        if len(normal_hs) > 2 and normal_hs not in web_words_content:
            final_result += normal_hs
    
    for path in path_split:
        normal_path = remove_extensions(path)
        normal_path = remove_special_characters(normal_path)
        if len(normal_path) > 2:
            final_result += normal_path
    
    return final_result


In [8]:
'''
Busca o arquivo de log e insere na base
'''

filepath = '../../Logs/dmoz-urlclassifier-master/dmoz.csv'

with open(filepath, 'r') as f:
    for line in f:
        splits = line.split(',')
        category = splits[-1].rstrip()
        url = splits[1]
        
        obj = {
            'url': url,
            'normalized_url': normalize_hostname(url),
            'category': category
        }
        
        db_log_lines.insert_one(obj)

In [9]:
#1562978